# Greedy Algorithm, Minimum Spanning Tree, and Dynamic Programming

## Application - internet routing

- ex. Stanford gateway router needs to send data to the Cornell gateway router
- Djikstra's algorithm does this (with nonnegative edge length)
- issue is that Stanford gateway router would need to know entire Internet
- need a shortest-path algorithm that uses only local computation
- solution is Bellman-Ford algorithm (also handles negative edge costs)

## Application - sequence alignment

- input: two strings over the alphabet {A,C,G,T}
- problem: figure out how similar the two strings are
- measure similarity via quality of "best" alignment
    - penalty $pen_{gap} \ge 0$ for each gap
    - penalty $pen_{AT} \ge 0$ for mismatching A and T
    - etc
- output: alignment of the strings that minimizes the total penalty (Needleman-Wunsch score)
- solution: straightforward dynamic programming

## Greedy algorithm

- ex. Dijkstra's shortest path algorithm
- easy to propose
- easy runtime analysis
- hard to eatablish correctness
- most greedy algorithms are not correct

## Application - optimal caching

- cache is faster than memory
- on a fault (cache miss), need to evict something from cache to make room
- theorem: the "furthest-in-future" algorithm is optimal (minimizes the number of cache misses)
- serves as guideline for practical algorithm ("Least Recently Used" should do well provided data exhibits locality of reference)
- serves as idealized benchmark for caching algorithms

## Application - scheduling

- setup
    - one shared resource(ex. a processor)
    - many "jobs" to do (ex. processes)
- question
    - in what order should we sequence the jobs?
- assume: each job has a 
    - weight $w_{j}$ ("priority")
    - length $l_{j}$
- definition: the completion time $c_{j}$ of job $j$ = sum of job lengths up to and including $j$
- goal: minimizes the weighted sum of completion times $min \displaystyle\sum_{j=1}^{n}w_{j}c_{j}$
- intuition
    - with equal lengths, schedule larger or smaller weight jobs earlier? larger
    - with equal weights, schedule shorter or longer jobs earlier? shorter
- what if $w_{i} \gt w_{j}$ but $l_{i} \gt l_{j}$?
    - assign "scores" to jobs that are
        - increasing in weight
        - decreasing in length
- guess #1: order jobs by decreasing value of $w_{j} - l_{j}$ (not always correct)
- guess #2: order jobs by decreasing raio $\dfrac{w_{j}}{l_{j}}$ (always correct, runs in $O(nlogn)$ - just need to sort)

### claim - guess #2 is alway correct

- by an exchange argument
- fix arbitrary input of $n$ jobs
- consider proof by contradiction
- let $\sigma$ = greedy schedule, $\sigma*$ = optimal schedule (with $\sigma*$ better than $\sigma$)
- assume all $\dfrac{w_{j}}{l_{j}}$'s are distinct
- assume by just renaming jobs $\dfrac{w_{1}}{l_{1}} \gt \dfrac{w_{2}}{l_{2}} \gt \dots \gt \dfrac{w_{n}}{l_{n}}$
- thus, greedy schedule $\sigma$ is just $1,2,3 \dots n$
- thus, if optimal schedule $\sigma^{*} \ne \sigma$, then there are consecutive jobs $i,j$ with $i>j$
- suppose we exchange order of $i,j$ in $\sigma^{*}$ (leaving other jobs unchanged)
    - cost of exchange is $w_{i}l_{j}$ ($c_{i}$ goes up by $l_{j}$)
    - benefit of exchange is $w_{j}l_{i}$ ($c_{j}$ goes down by $l_{i}$)
    - $i \gt j => \dfrac{w_{i}}{l_{i}} \lt \dfrac{w_{j}}{l_{j}} => \dfrac{w_{i}}{l_{j}} \lt \dfrac{w_{j}}{l_{i}}$
        - cost $\lt$ benefit, meaning swap improves $\sigma^{*}$, contradicts optimality of $\sigma^{*}$ 
        
### claim - guess #2 is correct even with ties

- fix arbitrary input of $n$ jobs
- let $\sigma$ = greedy schedule, $\sigma^{*}$ = any other schedule
- will show $\sigma$ at least as good as $\sigma^{*}$
    - implies that greedy schedule is optimal
- assume by just renaming jobs, greedy schedule $\sigma$ is just $1,2,3 \dots n$ (and so $\dfrac{w_{1}}{l_{1}} \gt \dfrac{w_{2}}{l_{2}} \gt \dots \gt \dfrac{w_{n}}{l_{n}}$)
- consider arbitrary schedule $\sigma*$. If $\sigma^{*} = \sigma$, done
- else recall there exists consecutive jobs $i,j$ in $\sigma^{*}$ with $i \gt j$
- exchanging $i$ and $j$ in $\sigma^{*}$ has net benefit of $w_{j}l_{i}-w_{i}l_{j} \ge 0$
- exchanging an "adjacent inversion" like $i,j$ only makes $\sigma^{*}$ better, and it decreases the number of inverted pairs (jobs $i,j$ with $i \gt j$ and $i$ scheduled earlier)
- after at most $n\choose{2}$ such exchanges, can transform $\sigma^{*}$ into $\sigma$
- $\sigma$ at least as good as $\sigma^{*}$
- greedy is optimal

## Minimum spanning trees

- input: "undirected" graph" $G=(V,E)$ and a cost (for each edge $e \in E$)
    - assume adjacency list representation 
    - OK if edge cost are negative
- output: minimum cost tree $T \subseteq E$ that spans all vertices 
    - $T$ has no cycles
    - subgraph $(U,T)$ is connected
- assumption #1: input graph $G$ is connected
    - else no spanning trees
    - easy to check in preprocessing (ex. depth-first search)
- assumption #2: edge costs are distinct
    - Prim + Kruskal remain correct with ties (which can be broken arbitrarily)
    
## Prim's MST algorithm

- runs in $O(mn)$
- initialize $X = \{s\}$ # $s \in V$ chosen arbitrary
- T = empty set # invariant: $X$ = vertices spanned by tree-so-far $T$
- while $X \ne V$ # increases the number of spanned vertices in cheapest way possible
    - let edge$(u,v)$ be the cheapest edge with $u \in X$ and $v \notin X$
    - add $e$ to $T$
    - add $v$ to $X$
    
### claim - Prim's algorithm outputs a spanning tree

- definition: a cut of a graph $G = (V,E)$ is a partition of $V$ into 2 non-empty sets
- empty cut lemma
    - a graph is not connected <=> there exists a cut$(A,B)$ with no crossing edges
    - proof: (<=)
        - assume RHS
        - pick any $u \in A$ and $v \in B$
        - since no edges cross $(A,B)$, there is no $u,v$ path in $G$ 
        - thus, $G$ not connected
    - proof: (=>)
        - suppose $G$ has no $u,v$ path
        - define $A$ = {vertices reachable from $u$ in $G$} ($u$'s connected component)
        - define $B$ = {all other vertices} (all other connected components)
        - note: no edges cross out $(A,B)$ (otherwise $A$ would be bigger!)
- double-crossing lemma
    - suppose the cycle $C \subseteq E$ has an edge crossign the cut$(A,B)$, then so does some other edge of $C$
- lonely cut corollary
    - if $e$ is the only edge crossing some cut$(A,B)$, then it is not in any cycle (if it were in a cycle, some other edge would have to cross the cut!)
- in summary
    - (1) algorithm maintains invariant that $T$ spans $X$
    - (2) can't get stuck with $X \ne V$ (otherwise the cut $(X, V-X)$ must be empty - by empty cut lemma, input graph $G$ is disconnected)
    - (3) no cycles ever get created in $T$
        - consider any iteration with current sets $X$ and $T$
        - suppose $e$ gets added
        - $e$ is the first edge crossing $(X, V-X)$ that gets added to $T$ => its addition can't create a cycle in $T$ (by lonely cut corollary)
        
### claim - Prim's algorithm always outputs a minimum-cost spanning tree

- cut property: consider an edge $e$ of $G$. suppose there is a cut $(A,B)$ such that $e$ is the cheapest edge of $G$ that crosses it. then $e$ belongs to the MST of $G$
- claim: cut property => Prim's algorithm is correct
    - already proved Prim's algorithm outputs a spanning tree $T^{*}$
    - key point: every edge $e \in T^{*}$ is explicitly justified by the cut property
        - $T^{*}$ is a subset of the MST
        - since $T^{*}$ is already a spanning tree, it must be the MST
- proof of cut property
    - suppose there is an edge $e$ that is the cheapest one crossing a cut$(A,B)$, yet $e$ is not in the MST $T^{*}$
    - idea: exchange $e$ with another edge in $T^{*}$ to make it even cheaper (contradiction)
    - since $T^{*}$ is connected, must construct an edge $f (\ne e)$ crossing $(A,B)$
    - idea: exchange $e$ and $f$ to get a spanning tree cheaper than $T^{*}$ (contradiction)
    - let $C$ = cycle created by adding $e$ to $T^{*}$
    - by double-crossing lemma: some other edge $e^{'}$ of $C$ (with $e^{'} \ne e$ and $e^{'} \in T^{*}$) crosses $(A,B)$ 
    - note: $T = T^{*}\cup\{e\}-\{e^{'}\}$ is also a spanning tree
    - since $C_{e} \lt C_{e^{'}}$, $T$ is cheaper than purported MST $T^{*}$, contradiction!
    
## Prim's algorithm with heaps

- invariant #1: elements in heap = vertices of $V-X$
- invariant #2: for $v \in V-X$, $key[v]$ = cheapest edge $(u,v)$ with $i \in X$ (or $+\infty$ if no such edges exist)
- check: can initialize heap with $O(m+nlogn) = O(mlogn)$ preprocessing
- note: given invariants, extract-min yields next vertex $v \notin X$ and edge $(u,v)$ crossing $(X, V-X)$ to add to $X$ and $T$, respectively
- issue: might need to recognize some keys to maintain invariant #2 after each extract-min

When $v$ added to $X$
- for each edge $(v,w) \in E$
    - if $w \in V-X$
        - (update key if needed)
        - delete $w$ from heap
        - recompute $key[w] = min[key[w], c_{vw}]$
        - re-insert into heap
        
Running time with heaps
- dominated by time required for heap operations
- $(n-1)$ inserts during preprocessing
- $(n-1)$ extract-mins (one per iteration of while loop)
- each edge $(v,w)$ triggers one delete/insert combo (when its first endpoint is sucked into $X$)
- $O(m)$ heap operations (recall $m \ge n-1$ since $G$ connected)
- $O(mlogn)$ time

In [17]:
def open_file(file_path):
    """
    Read-in a file containing rows of data
    
    Args:
    file_path (string) -- location of file to read
    
    Returns:
    tuple_data (tuple of list and integer) -- adjancency representation of graph and number of nodes
    """
    
    data_array = []
    num_nodes = 0
    
    with open(file_path, 'r') as line:
        array_of_array = line.read().split("\n")
        num_nodes = int(array_of_array[0].split(" ")[0]) 
        del array_of_array[0] # delete first element, which is just the length of data
        for array in array_of_array:
            subarray = array.split(" ")
            node1 = int(subarray[0])
            node2 = int(subarray[1])
            cost = int(subarray[2])
            data_array.append((node1, node2, cost))
            
    tuple_data = (data_array, num_nodes)
    return tuple_data


def greedy_search(array, X, T):
    """
    For all node1 in X, find node2 that is not in X, that makes the cheapest edge between node1 and node2
    
    Args:
    array (list) -- adjancency representation of graph
    X (list) -- stores all vertices that consist minimun spanning tree
    T (list) -- stores all costs of edges that consist minimun spanning tree
    
    Returns:
    None
    """
    
    minimum_cost = 1000000
    minimum_node1 = 0
    minimum_node2 = 0
    for node1 in X:
        for node2 in get_connected_node(node1, array):
            if node2 not in X:
                cost = get_cost(node1, node2, array)
                if cost < minimum_cost:
                    minimum_node1 = node1
                    minimum_node2 = node2
                    minimum_cost = cost
    
    X.append(minimum_node2)
    T.append(minimum_cost)
    
    
def get_connected_node(node1, array):
    """
    Find all nodes that are connected by an edge for node1
    
    Args:
    node1 (integer) -- input node
    array (list) -- adjancency representation of graph
    
    Returns:
    nodes (list) - all nodes connected to node1
    """
    
    nodes = []
    
    for item in array:
        if item[0] == node1:
            nodes.append(item[1])
        elif item[1] == node1:
            nodes.append(item[0])
            
    return nodes


def get_cost(node1, node2, array):
    """
    Find cost of edge between node1 and node2
    
    Args:
    node1 (integer) -- first node of an edge
    node2 (integer) -- second node of an edge
    array (list) -- adjancency representation of graph
    
    Returns:
    cost (integer) -- cost of edge between node1 and node2
    """
    
    cost = 0
    
    for item in array:
        if item[0] == node1 and item[1] == node2:
            cost = item[2]
        if item[0] == node2 and item[1] == node1:
            cost = item[2]
            
    return cost


def prim(file_path):
    """
    Read-in a file containing rows of data
    
    Args:
    file_path (string) -- location of file to read
    
    Returns:
    cost (integer) -- cost of minimum spanning tree
    """
    
    tuple_obj = open_file(file_path)
    array = tuple_obj[0]
    num_nodes = tuple_obj[1]
    
    X = [] # store explored nodes
    s = array[0][0] # pick random node
    X.append(s)
    
    T = [] # store costs
    T.append(0)
    
    while len(X) < num_nodes:
        greedy_search(array, X, T)
    
    cost = sum(T)
    return cost

In [18]:
assert(prim("data/edge.txt") == -3612829)
assert(prim("data/edge1.txt") == 7)
assert(prim("data/edge2.txt") == 15)
assert(prim("data/edge3.txt") == 14)

## MST review

- input: undirected graph $G = (V,E)$, edge cost $c_{e}$
- output: min-cost spanning tree (no cycles, connected)
- assumptions: $G$ is connected, distinct edge costs
- cut property: if $e$ is the cheapest edge crossing some cut$(A,B)$, then $e$ belongs to the MST


## Kruskal's MST Algorithm 

- $O(mn)$
- sort edges in order of increasing cost (rename edges 1,2,3,... so that $c_{1} < c_{2} < \dots < c_{m}$)
- let $T$ = empty set
- for $i=1 \dots m$ # $O(m)$
    - if $T\cup\{i\}$ has no cycles
        - add $i$ to $T$ # $O(n)$ use BFS/DFS in the graph $(V,T)$ which contains $\le n-1$ edges  
- return $T$

### correctness

- let $T^{*}$ = output of Kruskal's algorithm on input graph $G$
- clearly $T^{*}$ has no cycles
- $T^{*}$ is connected
    - by empty cut lemma, only need to show that $T^{*}$ crosses every cut
    - fix a cut$(A,B)$, since $G$ connected at least one of its edges cross $(A,B)$
- key point: Kruskal will include first edge crossing $(A,B)$ that it sees (by lonely cut corollary, cannot create a cycle)
- every edge of $T^{*}$ satisfied by the cut property (implies $T^{*}$ is the MST)
    - consider iteration where edge $(u,v)$ added to current set $T$. since $T\cup\{(u,v)\}$ has no cycle, $T$ has no $u-v$ path
        - there exists an empty cut$(A,B)$ separating $u$ and $v$ (as in proof of empty cut lemma)
        - no edges crossing $(A,B)$ were previsouly considered by Kruskal's algorithm
        - $(u,v)$ is the first (hence the cheapest!) edge crossing $(A,B)$
        - $(u,v)$ justified by the cut property

## Union-Find data structure

- maintain partition of a set of objects
- Find$(x)$: return name of group that $x$ belongs to
- Union$(c_{i},c_{j})$: fuse groups $c_{i},c_{j}$ into a single one

### why usefu for Kruskal's?

- objects = vertoces
- groups = connected components w.r.t. chosen edges $T$
- adding new edge $(u,v)$ to $T$ <=> fusing connected components of $u,v$

### Union-Find basics

- motivation: $O(1)$ time cycle checks in Kruskal's algorithm
- idea #1: maintain one linked structure per connected component of $(V,T)$
    - each component has an arbitrary leader vertex
- invariant: each vertex points to the leader of its component ("name" of a component inherited from leader vertex)
- key point: given edge$(u,v)$, can check if $u$ and $v$ already in same component in $O(1)$ time (iff leader pointers of $u$ and $v$ match <=> Find$(u)$ = Find$(v)$ => $O(1)$ time cycle checks!)
- note: when new edge $(u,v)$ added to $T$, connected components of $u$ and $v$ merge
- how many times does a single vertex $v$ have its leader pointer updated over the course of Kruskal's algorithm?
    - $O(logn)$ because every time $v$'s leader gets updated, population of its component at least doubles => can only happen $\le log_{2}^n$ time

### Running time

- $O(mlogn)$ for sorting
- $O(m)$ for cycle checks ($O(1)$ per iteration)
- $O(nlogn)$ overall for leader pointer updates
- $O(mlogn)$ total, matching Prim's

### State-of-the-art MST

- $O(m)$ randomized algorithm (Karger-Klein-Tarjan JACM 1995)
- $O(m\alpha(n))$ deterministic (Chazelle JACM 2000)
    - "ïnverse Ackerman function": grows much slower than $log^{*}n$

## Clustering

- "unsupervised learning"
- informal goal: given $n$ points, classify into "coherent groups"
- assumptions
    - as input, given a (dis)similarity measure - a distance $d(p,g)$ between each point pair
    - symmetric (ex. $d(p,g) = d(g,p)$)
- ex. Euclidean distance, genome similarity, etc

### Max-spacing k-clusterings

- assume: we know $k$ = number of clusters desired (in practice, can experiment with a range of values)
- call pointers $p,q$ separated if they are assigned to different clusters
- definition: the spacing of a $k$-clustering in $min_{separated\ p,q}d(p,q)$ (bigger the better)
- problem: given a distance measure $d$ and $k$, compute the $k$-clustering with maximum spacing

### A greedy algorithm

- initially, each point in a separate cluster
- repeat until only $k$ clusters
    - let $p,q$ = closest paif of separate points (determines the current spacing)
    - merge the cluster containing $p$ and $q$ into a single cluster
- just like Kruskal's MST, but stopped early (single-link clustering)
    - points <=> vertices
    - distances <=> edge costs
    - point pairs <=> edges
    
### Correctness

- claim: single-link clustering finds the max-spacing $k$-clustering 
- proof
    - let $c_{1} \dots c_{k}$ = greedy clustering with spacing $S$
    - let $\hat{c_{1}} \dots \hat{c_{k}}$ = arbitrary other clustering
    - need to show spacing of $\hat{c_{1}} \dots \hat{c_{k}}$ is $\le S$
    - case #1: $\hat{c_{i}}$'s are the same as the $c_{i}$'s (maybe after remaning) => has the same spacing $S$
    - case #2: otherwise, can find a point pair $p,q$ such that 
        - $p,q$ in the same greedy cluster $c_{i}$
        - $p,q$ in different clusters $\hat{c_{i}},\hat{c_{j}}$
    - property of greedy algorithm: if two points $x,y$ "directly merged at some point", then $d(x,y) \le S$ (distance between merged point pairs only goes up)
    - easy case: if $p,q$ directly merged at some point, $S \ge d(p,q) \ge$ spacing of $\hat{c_{1}} \dots \hat{c_{k}}$
    - tricky case: $p,q$ "indirectly merged" through multiple direct merges
        - let $p,a_{1} \dots a_{l},q$ be the path of direct greedy merges connecting $p$ and $q$
        - key point: since $p \in \hat{c_{i}}$ and $q \notin \hat{c_{i}}$, $\exists$consecutive pair $a_{j}, a_{j+1}$ with $a_{j} \in \hat{c_{i}}, a_{j+1} \notin \hat{c_{i}}$ => $s \ge d(a_{j}, a_{j+1}) \ge$ spacing of $\hat{c_{1}} \dots \hat{c_{k}}$
        
## Advanced Union-Find

### Previous solution (for Kruskal's MST)

- each $x \in X$ points directly to the "leader" of its group
- $O(1)$ Find (just return $x$'s leader)
- $O(nlogn)$ total works for $n$ Unions (when 2 groups merge, smaller group inherits leader of larger one)

### Lazy Union

- new idea: update only one pointer each merge!
- in general: when two groups merge in a Union, make one group's leader (ex. root of the tree) a child of the other one
- pro: Union reduces to 2 Finds ($r_{1}$ = Find$(x)$, $r_{2}$ = Find$(y)$) and $O(1)$ extra work (link $r_{1}, r_{2}$ together)
- con: to recover leader of an object, need to follow a pth of parent pointers (not just one!) => not clear if Find still takes $O(1)$
- new implementation: each object $x \in X$ has a parent field 
- invariant: parent pointers induce a collection of directed trees on $x$ ($x$ is root <=> parent$[x] = x$)
- initially: for all $x$, parent$[x] = x$
- Find$(x)$: traverse parent pointers from $x$ until you hit the root
- Union$(x,y)$: $s_{1}$ = Find$(x)$; $s_{2}$ = Find$(y)$. reset parent of one of $s_{1}, s_{2}$ to be the other

### Union by rank

- for each $x \in X$, maintain field rank$[x]$ (in general rank$[x] = 1 + $(max rank of $x$'s children))
- invariant: for all $x \in X$, rank$[x]$ - maximum number of hops from some leaf to $x$ (initially, rank$[x] = 0$ for all $x \in X$)
- to avoid scraggly trees, given $x$ and $y$
    - $s_{1}$ = Find$(x)$, $s_{2}$ = Find$(y)$
    - if rank$[s_{1}]$ $\gt$ rank$[s_{2}]$, then set parent$[s_{2}]$ to $s_{1}$, else get parent$[s_{1}]$ to $s_{2}$ 
- make old root with smaller rank child of the root with the larger rank (choose new root arbitrarily in case of a tie and add $1$ to its rank)    

### Properties of rank

- immediate from invariant/rank maintenance
    - for all objects $x$, rank$[x]$ only goes up over time
    - only rank of roots can go up (once $x$ a non-root, rank$[x]$ forzen forevermore)
    - ranks strictly increase along a path to the root
    
### Rank lemma

- consider an arbitrarty sequence of Union (+ Find) operations. For every $r \in {0,1,2,\dots}$, there are at most $n/2^{r}$ objects with rank $r$
- corollary: max rank always $\le log_{2}n$
- corollary: worst-case running time of Find, Union is $O(logn)$
- claim: if $x,y$ have the same rank $r$, then their subtrees (objects from which can reach $x,y$) are disjoint
- proof
    - suppose subtrees of $x,y$ have object $z$ in common
        - $\exists$paths $z->x, z->y$
        - one of $x,y$ is an ancester of the other
        - the ancestor has strictly larger rank
- claim: the subtree of a rank $r$ object has size $\ge 2^{r}$
- proof
    - rank $r$ => subtree size $\ge 2^{r}$
    - base case: initialy all ranks $= 0$, all subtree sizes $= 1$
    - inductive step: nothing to prove unless the rank of some object changes (subtree sizes only go up)
    - interesting case: Union$(x,y)$, with $s_{1}=$ Find$(x)$, $s_{2}=$ Find$(y)$, and rank$[s_{1}] =$ rank$[s_{2}] = r$ => $s_{2}$'s new rank $= r+1$ => $s_{2}$'s new subtree size $= s_{2}$'s old subtree size $+ s_{1}$'s old subtree size (each at least $2^{r}$ by the inductive hypothesis) $\ge 2^{r+1}$ 
    
### Path compression

- idea: why bother traversing a leaf-root path multiple-times? after Find$(x)$, install shortcuts (ex. revise parent pointers) to $x$'s root all along the $x$ => root path
- con: constant-factor overhead to Find (from "multitasking")
- pro: speeds up subsequent Finds

### On ranks
- important: maintain all rank fields exactly as without path compression
    - rank initially all 0
    - in Union, new root = old root with bigger rank
    - when merging two nodes of common rank $r$, reset new root's rank to $r+1$
- bad news: now rank$[x]$ is only an upper boud on the maximum number of hops on a path from a leaf to $x$ (which could be much less)
- good news: rank lemma still holds ($\le n/2^{r}$ objects with rank $r$) still always have rank$[$parent$[x]]$ > rank$[x]$ for all non-roots $x$

### Hopcroft-Ullman theorem

- with union by rank and path compression, $m$ Union + Find operations take $O(mlog^{*}n)$ time, where $log^{*}n$ = the number of times you need to apply $log$ to $n$ before the result is $\le 1$

### Measuring progress

- initution: installing shortcuts should significantly speed up subsequent Finds + Unions
- question: how to track this progress and quantify the benefit? 
- idea: consider a non-root object $x$
    - progress measre: rank$[$parent$[x]]$ - rank$[x]$
- path compression increases this progress measure: if $x$ has old parent $p$, new parent $p' \ne p$, then rank$[p^{'}] \gt$rank$[p]$

### Proof setup

- rank blocks: $\{0\},\{1\},\{2,3,4\},\{5 \dots 2^{4}\},\{17,18 \dots 2^{16}\},\{65537 \dots 2^{65536}\} \dots \{\dots n\}$
- note: there are $O(log^{*}n)$ different rank blocks
- semantics: traversal $x$ -> parent$(x)$ is "fast progress" <=> rank$[$parent$[x]]$ is larger block than rank$[x]$
- definition: at a given point in time, call object $x$ "good" if 
    - $x$ or $x$'s parent is a root OR
    - rank[parent$[x]$] in larger block than rank$[x]$
    
### Proof of Hopcroft-Ullman

- point: every Find visits only $O(log^{*}n)$ good nodes $(2 + $number of rank blocks = $O(log^{*}n)$ $)$
- upshot: total work done during $m$ operations = $O(mlog^{*}n)$ (visits to good objects) + total number of visits to bad nodes (need to bound globally by separate argument)
- consider: a rank block $\{k+1, k+1 \dots 2^{k}\}$
- note: when a bad node is visited, its parent is changed to one with strictly larger rank => can only happen $2^{k}$ times before $x$ becomes good (forevermore)
- rank lemma: total number of objects $x$ with final rank in this rank block is $\displaystyle\sum_{i=k+1}^{2^{k}}n/2^{i} \le n/2^{k}$
- recall: only $O(log^{*}n)$ rank blocks
- total work: $O((m+n)log^{*}n)$

### Tarjan's bound

- theorem: with union by rank and path compression, $m$ Union + Find operations take $O(m\alpha(n))$ time, where $\alpha(n)$ is the inverse Ackerman function

### Ackerman function

- define $A_{k}(r)$ for all integers $k$ and $r \ge 1$ (recursively)
- base case: $A_{0}(r) = r+1$ for all $r \ge 1$
- in general, for $k,r \ge 1$
    - $A_{k}(r)$ = apply $A_{k-1}(r)$ times to $r = (A_{k-1} \circ A_{k-1} \circ \dots \circ A_{k-1})(r)$
    
### Inverse Ackerman function

- definition: for every $n \ge 4, \alpha(n)$ = minimum value of $k$ such that $A_{k}(2) \ge n$

### Building blocks of Hopcroft-Ullman analysis

- block #1: rank lemma (at most $n/2^{r}$ objects of rank $r$)
- block #2: path compression => If $x$'s parent pointer updated from $p$ to $p'$, then rank$(p')$ $\ge$ rank$(p)+1$
- new idea: stronger version of building block #2. in most cases, rank of new parent much bigger than rank of old parent (not just by 1)

### Quantifying rank gaps

- definition: consider a non-root object $x$ (so rank$[x]$ fixed forevermore)
- define: $\delta(x)$ = max value of $k$ such that rank$[$parent$[x]] \ge A_{k}($rank$[x])$ 
- ex. always have $\delta(x) \ge 0$
    - $\delta(x) \ge 1$ <=> rank$[$parent$[x]] \ge 2$rank$[x]$
    - $\delta(x) \ge 2$ <=> rank$[$parent$[x]] \ge $rank$[x]2^{rank[x]}$
for all objects $x$ with rank$[x] \ge 2$, then $\delta(x) \le \alpha(n)$ (since $A_{\alpha(n)}(2) \le n$) 

### Bad objects

- definition: an object is bad if all of the following holds
    - $x$ is not a root
    - parent$(x)$ is not a root
    - rank$(2) \ge 2$
    - $x$ has an ancestor $y$ with $\delta(y) = \delta(x)$
    
### Proof of Tarjan's bound

- upshot: total work of $m$ operations = $O(m\alpha(n))$ (visits to good objects) + total number of visits to bad objects (will show is $O(n\alpha(n))$)
- main argument: suppose a Find operation visits a bad object $x$
- path compression: $x$'s new parent will be $p^{'}$ or even higher
    - rank$[x$'s new parent$] \ge$ rank$[p^{'}] \ge A_{k}($rank$[y]) \ge A_{k}($rank$[p])$ 
- point: path compression (at least) applies the $A_{k}$ function to rank$[x$'s parent$]$
- consequence: if $r = $rank$[x] (\ge 2)$, then after $r$ such pointer updates we have 
    - rank$[x$'s parent$] \ge (A_{k} \circ \dots r$ times $ \dots \circ A_{k})(r) = A_{k+1}(r)$
- thus, while $x$ is bad, every $r$ vistis increases $\delta(x)$
    - $\le r\alpha(n)$ visits to $x$ while it's bad
- total number of visits to bad objects $\le \displaystyle\sum_{objects\ x}$ rank$[x]\alpha(n) = \alpha(n)\displaystyle\sum_{r \ge 0}r$ (number of objects with rank $r$) = $n\alpha(n)\displaystyle\sum_{r \ge 0} r/2^{r} = O(n\alpha(n))$ 

In [1]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path -- location of file to read

    Returns:
    (data_array, num_nodes) -- a tuple with an array representing a graph and an integer reprsenting number of nodes
    """

    data_array = []
    num_nodes = 0

    with open(file_path, 'r') as line:
        array_of_array = line.read().split("\n")
        num_nodes = int(array_of_array[0].split(" ")[0])
        del array_of_array[0] # delete first element, which is just the length of data
        for array in array_of_array:
            subarray = array.split(" ")
            node1 = int(subarray[0])
            node2 = int(subarray[1])
            cost = int(subarray[2])
            data_array.append((node1, node2, cost))
    return (data_array, num_nodes)


def find_closest_pair_and_merge(sorted_array, T):
    """
    Find two nodes that are in different clusters, and merge them into a single cluster

    Args:
    sorted_array -- a list of tuple what is sorted by its thrid element (that is cost between two nodes)
    T -- a list of list that contains "clusers"

    Returns:
    None
    """

    node1 = sorted_array[0][0]
    node2 = sorted_array[0][1]
    cost = sorted_array[0][2]

    index_of_cluster_to_expand = find_cluster(node1, T)
    index_of_cluster_to_remove = find_cluster(node2, T)

    print(str(node1) + " and " + str(node2) + ": " + str(index_of_cluster_to_expand) + " => " + str(index_of_cluster_to_remove))

    if index_of_cluster_to_expand != index_of_cluster_to_remove: # if two nodes are already in the same cluster, no need to perform merge on T
        for node in T[index_of_cluster_to_remove]:
            T[index_of_cluster_to_expand].append(node) # add all nodes in the cluster where node2 belongs to node1's cluster
        del T[index_of_cluster_to_remove] # remove node2's cluster
        del sorted_array[0] # remove current tuple
    else:
        del sorted_array[0] # remove current tuple


def find_cluster(node, T):
    """
    Find a list inside T where node belongs

    Args:
    node -- an integer representing a node in a graph
    T -- a list of list that contains "clusers"

    Returns:
    i -- index of cluster of T
    """

    for i in range(0, len(T)):
        if node in T[i]:
            return i
    return -1


def get_max_spacing(T, sorted_array):
    """
    Return the minimum distance of two nodes that are in different clusters

    Args:
    sorted_array -- a list of tuple what is sorted by its thrid element (that is cost between two nodes)
    T -- a list of list that contains "clusers"

    Returns:
    item[2] -- the minimum cost
    """

    for item in sorted_array:
        cluster_of_node1 = find_cluster(item[0], T)
        cluster_of_node2 = find_cluster(item[1], T)
        if cluster_of_node1 != cluster_of_node2:
            return item[2]


tuple_obj = open_file("data/clustering.txt")
# tuple_obj = open_file("data/clustering-test1.txt")
array = tuple_obj[0]
sorted_array = sorted(array, key=lambda x: (x[2])) # sort by third element
num_nodes = tuple_obj[1]
print("len(array):" + str(len(sorted_array)))


T = []
for node in range(1, num_nodes+1):
    T.append([node])
print("len(T): " + str(len(T)))
print(T)

while len(T) > 4 and len(sorted_array) > 0:
    find_closest_pair_and_merge(sorted_array, T)

print(get_max_spacing(T, sorted_array))

# Max-spacing:100, two clusters: Nodes(1,2) Nodes(3,4,5)
# Max-spacing:105, four clusters
# Max-spacing:106

len(array):124750
len(T): 500
[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119], [120], [121], [122], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [136], [137], [138], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [152], [153], [154],

59 and 267: 0 => 0
59 and 280: 0 => 0
74 and 233: 0 => 0
89 and 239: 0 => 0
91 and 366: 0 => 0
92 and 137: 0 => 0
148 and 362: 0 => 0
232 and 475: 0 => 0
256 and 421: 0 => 0
334 and 344: 0 => 0
41 and 421: 0 => 0
48 and 268: 0 => 0
76 and 301: 0 => 0
93 and 353: 2 => 0
103 and 195: 1 => 1
117 and 143: 1 => 1
180 and 256: 1 => 1
205 and 332: 1 => 1
211 and 347: 1 => 1
266 and 273: 1 => 1
308 and 473: 1 => 1
17 and 66: 1 => 1
20 and 108: 1 => 1
21 and 407: 1 => 1
39 and 116: 1 => 1
52 and 189: 1 => 1
67 and 303: 1 => 1
71 and 129: 1 => 1
116 and 293: 1 => 1
174 and 399: 1 => 1
191 and 456: 1 => 1
218 and 247: 1 => 1
227 and 471: 1 => 1
304 and 422: 1 => 1
322 and 470: 1 => 1
425 and 497: 1 => 1
24 and 38: 1 => 1
42 and 355: 1 => 1
55 and 91: 1 => 1
55 and 423: 1 => 1
95 and 440: 1 => 1
170 and 382: 1 => 1
259 and 335: 1 => 1
355 and 399: 1 => 1
17 and 201: 1 => 1
25 and 270: 1 => 1
26 and 246: 1 => 1
72 and 431: 1 => 1
129 and 450: 1 => 1
135 and 424: 1 => 1
166 and 314: 1 => 1
176 and 1

In [2]:
from networkx.utils.union_find import UnionFind


def open_file(file_path):
    """
    Read-in a file containing rows with weight and length, and compute difference and ratio

    Args:
    file_path -- location of file to read

    Returns:
    data_array -- an array of tuplesrepresenting a graph
    """

    data_dict = {}
    data_array = []
    num_nodes = 0

    with open(file_path, 'r') as line:
        array_of_array = line.read().split("\n")
        num_nodes = int(array_of_array[0].split(" ")[0])
        num_bits = int(array_of_array[0].split(" ")[1])
        del array_of_array[0] # delete first element, which is just metadata
        for i in range(0, len(array_of_array)):
            number = int(array_of_array[i].replace(" ", ""))
            data_array.append(number)
            if number not in data_dict:
                data_dict[number] = set()
            data_dict[number].add(i+1)
                  
    return (data_array, data_dict, num_nodes, num_bits)


def convert_base_10_to_2(array):
    """
    Convert a list of integers (base 10) to a list of integers (base 2)
    
    Args:
    array - list of integers
    
    Returns:
    None
    """
    for i in range(0, len(array)):
        array[i] = int(bin(array[i])[2:])
    
    
tuple_obj = open_file("data/clustering-big.txt")
# tuple_obj = open_file("data/clustering-big-test1.txt")
# tuple_obj = open_file("data/clustering-big-test2.txt")
data_array = tuple_obj[0]
data_dict = tuple_obj[1]
num_nodes = tuple_obj[2]
num_bits = tuple_obj[3]
print("len(data_array): " + str(len(data_array)))
print("len(data_dict): " + str(len(data_dict)))
print("num_nodes: " + str(num_nodes))
print("num_bits: " + str(num_bits))

unionFind = UnionFind()

# Hemming distance of 1
heming_distance_1 = [1 << i for i in range(num_bits)]

convert_base_10_to_2(heming_distance_1)
print(len(heming_distance_1)) #24

# Hemming distance of 2
heming_distance_2 = []
for i in range(0, len(heming_distance_1)):
    for j in range(0, len(heming_distance_1)):
        if j > i:
            dist = int(str(heming_distance_1[i]),2) ^ int(str(heming_distance_1[j]),2)
            heming_distance_2.append(dist)
        
convert_base_10_to_2(heming_distance_2)
print(len(heming_distance_2)) # 276

distances = heming_distance_1 + heming_distance_2
print(len(distances))

for distance in distances:
    for key1 in data_dict:
        key2 = int(str(distance),2) ^ int(str(key1),2)
        key2 = int(bin(key2)[2:]) 
        if key2 in data_dict:
            unionFind.union(key1, key2)

pointer_set = set([unionFind[x] for x in data_dict])
num_clusters = len(pointer_set)
print(num_clusters)

# 3
# 15
# 6118

len(data_array): 200000
len(data_dict): 198788
num_nodes: 200000
num_bits: 24
24
276
300
6118


# Huffman Codes

Binary code: maps alphabet to binary string. For example, {A, B, C, D} => {00, 01, 10, 11}
How about use instead "prefix-free" such that {A, B, C, D} => {0, 10, 110, 111}

In general
- left child edges get "0"
- right child edges get "1"
- for each $i$, there is exactly one node labelled $i$
- encoding: bits along path from root to node $i$
- decoding: repeatedly follow path from root until hitting a leaf
- encoding length of $i$ = depth of $i$ in a tree

Given probability $p_{i}$ for each character $i$, find Tree $T$ that minimize the length of encoding defined by

$L(T) = \displaystyle\sum_{i}P_{i}$(depth of $i$ in T)

Idea: build the tree bottom up using successive merges
- if len(set) = 2, return
- let $a$,$b$ have the smallest frequencies
- let new_set = set with $a$ & $b$ replaced by $ab$
- define $p_{ab} = p_{a} + p_{b}$ 
- recursively comput $T^{'}$ (for new_set)
- extend $T^{'}$ to $T$ by splitting leaf $ab$ into two leave $a$ & $b$
- return $T$

In [3]:
import itertools 


def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path -- location of file to read

    Returns:
    (data_dict, num_nodes) -- a tuple with a dictionary representing a graph and an integer reprsenting number of nodes
    """

    data_dict = {}
    num_nodes = 0
    index = 1

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_nodes = int(data_array[0].split(" ")[0])
        del data_array[0] # delete first element, which is just the length of data
        for item in data_array:
            data_dict[str(index)] = int(item)
            index += 1
    return (data_dict, num_nodes)


tuple_obj = open_file("data/huffman.txt")
# tuple_obj = open_file("data/huffman-test1.txt")
data_dict = tuple_obj[0]
num_nodes = tuple_obj[1]

sorted_dict_by_value = {k: v for k, v in sorted(data_dict.items(), key=lambda item: item[1])}
tree_merge_track = []

while len(sorted_dict_by_value) > 2:
    first_two_items = dict(itertools.islice(sorted_dict_by_value.items(), 2)) # get two smallest values
    first_node = ""
    second_node = ""
    new_weight = 0
    for key, value in first_two_items.items():
        if first_node == "":
            first_node = key
        else:
            second_node = key
        new_weight += value
        del sorted_dict_by_value[key] # delete two smallest nodes
        
    new_node = first_node + " " + second_node 
    tree_merge_track.append(new_node) 
    sorted_dict_by_value[new_node] = new_weight # create a new node that is a combination of the two smallest nodes
    sorted_dict_by_value = {k: v for k, v in sorted(sorted_dict_by_value.items(), key=lambda item: item[1])}
    
# print(sorted_dict_by_value)

# Find "occurance" of each node in merge operation
count_dict = {}
for item in tree_merge_track:
    for char in item.split(" "):
        if char not in count_dict:
            count_dict[char] = 1
        count_dict[char] += 1
        
print(len(count_dict))
print(count_dict)
sorted_count_dict_by_value = {k: v for k, v in sorted(count_dict.items(), key=lambda item: item[1])}
# Max: 19, Min: 9

1000
{'472': 19, '799': 19, '753': 18, '449': 17, '555': 16, '868': 16, '357': 16, '958': 16, '599': 16, '383': 16, '211': 16, '539': 16, '372': 16, '805': 15, '879': 15, '972': 15, '175': 15, '757': 15, '942': 15, '471': 15, '622': 15, '988': 15, '312': 15, '849': 15, '926': 15, '927': 15, '314': 15, '947': 15, '818': 14, '277': 14, '962': 14, '486': 14, '67': 14, '701': 14, '597': 14, '29': 14, '811': 14, '937': 14, '491': 14, '305': 14, '713': 14, '806': 14, '249': 14, '881': 14, '453': 14, '246': 14, '117': 13, '976': 13, '636': 13, '620': 13, '931': 13, '368': 13, '675': 13, '384': 13, '667': 13, '26': 13, '603': 13, '193': 13, '189': 13, '726': 13, '110': 13, '557': 13, '88': 13, '120': 13, '964': 13, '596': 13, '589': 13, '474': 13, '732': 13, '717': 13, '477': 13, '802': 13, '913': 13, '953': 13, '297': 13, '50': 13, '645': 13, '264': 13, '820': 13, '997': 13, '351': 13, '207': 13, '107': 13, '689': 13, '97': 13, '985': 13, '15': 13, '986': 13, '705': 13, '671': 13, '28': 13, '

# Dynamic Programming

Ex. Graph G = (V,E) with non-negative weights on vertices. Compute subset of non-adjacent vertices that constitute the maximum total weight

Let $S$ (in $V$) be a max-weight independent set
- suppose $v_{n}$ not in $S$
- let $G^{'}$ = $G$ with $v_{n}$ deleted
- S is also an independent set of $G^{'}$
- S must be a max-weight independent set of $G^{'}$

This time
- suppose $v_{n}$ in $S$
- then, previous vertex $v_{n-1}$ not in $S$
- let $G^{''}$ = $G$ with $v_{n}$ and $v_{n-1}$ deleted
- S-{$v_{n}$} is also an independent set of $G^{'}$
- S-{$v_{n}$} must be a max-weight independent set of $G^{''}$

Thus, max-weight independent set must be either
- max-weight independent set of $G^{'}$ or
- $v_{n}$ + max-weight independent set of $G^{''}$

Algorithm
- let $G_{i}$ = 1st $i$ vertices of $G$
- populate array $A$ left to right with $A[i]$ = value of max-weight independent set of $G_{i}$
- init: $A[0] = 0$ and $A[1] = w_{1}$
- main loop: for $i = 2,3,4 \dots n$, $A[i] = max[A[i-1], A[i-2]+w_{i}]$

Then trace back through filled-in array to reconstruct optimal solution
- let $A$ = filled-in array
- let $S$ = empty set
- while $i \ge 1$ 
    - if $A[i-1] \ge A[i-2] + w_{i}$
        - decrease i by 1
    - else
        - add $v_{i}$ to $S$ 
        - decrease $i$ by 2
- return $S$

### Principle of Dynamic Programming
1. Identify a small number of sub-problems
2. Given solutions to smaller sub-problems, can solve larger sub-problems
3. Solving all sub-problems computes final solution

In [4]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path -- location of file to read

    Returns:
    (data_dict, num_nodes) -- a tuple with a dictionary representing a graph and an integer reprsenting number of nodes
    """

    data_dict = {}
    num_nodes = 0
    index = 1

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_nodes = int(data_array[0].split(" ")[0])
        del data_array[0] # delete first element, which is just the length of data
        for item in data_array:
            data_dict[index] = int(item)
            index += 1
    return (data_dict, num_nodes)


tuple_obj = open_file("data/max-weight-independent-set.txt")
# tuple_obj = open_file("data/max-weight-independent-set-test1.txt")
# tuple_obj = open_file("data/max-weight-independent-set-test2.txt")
data_dict = tuple_obj[0]
num_nodes = tuple_obj[1]

A = {}
A[0] = 0
A[1] = data_dict[1]
for i in range(2, num_nodes + 1):
    A[i] = max(A[i-1], A[i-2] + data_dict[i])

S = set()
while num_nodes > 1:
    if A[num_nodes-1] >= A[num_nodes-2] + data_dict[num_nodes]:
        num_nodes -= 1
    else:
        S.add(num_nodes)
        num_nodes -= 2
if 2 not in S:
    S.add(1)

ret = ""
for i in [1, 2, 3, 4, 17, 117, 517, 997]:
    if i in S:
        ret += "1"
    else:
        ret += "0"
print(ret)
# 10100110

10100110


## Knapsack Problem

Ex. n items
- value $v_{i}$ (non-negative)
- size $w_{i}$ (non-negative and integral)
- capacity $W$ (non-negative integer)

Find subset $S$ in ${1 \dots n}$ that maximizes $\displaystyle\sum_{i}v_{i}$ subject to $\displaystyle\sum_{i}w_{i} \le W$

Let S = a max-value solution
- suppose item n not in $S$. Then $S$ must be optimal with first $n-1$ items with capacity $W$
- suppose item n in $S$. Then $S-\{n\}$ must be optimal with first $n-1$ items with capacity $W-w_{n}$

Let $v_{i,x}$ = value of the best solution that
- uses only the first $i$ items
- has total size $\le x$

Then,
- for i = 1 to n and any x
    - $v_{i,x}$ = max{$v_{i-1,x}$ (case when item $i$ in excluded), $v_{i} + v_{i-1,x-w_{i}}$ (case when item $i$ in included)}
- if $w_{i} > x$, then $v_{i,x} = v_{i-1,x}$

Pseudo code
- let A = 2-D array
- init $A[0,x] = 0$ for $x = 0 \dots W$
- for i = 1 to n
    - for $x = 0 \dots W$
        - $A[i,x] = max\{A[i-1, x], A[i-1, x-w_{i}] + v_{i}\}$
- return $A[n,W]$

In [5]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path -- location of file to read

    Returns:
    (data_dict, num_nodes) -- a tuple with a dictionary representing a graph and an integer reprsenting number of nodes
    """

    data_dict = {}
    knapsack_size = 0
    num_items = 0
    index = 1

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        knapsack_size = int(data_array[0].split(" ")[0])
        num_items = int(data_array[0].split(" ")[1])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            value = int(item.split(" ")[0])
            weight = int(item.split(" ")[1])
            data_dict[index] = (value, weight)
            index += 1
    return (data_dict, knapsack_size, num_items)


# tuple_obj = open_file("data/knapsack-test1.txt")
# tuple_obj = open_file("data/knapsack-test2.txt")
# tuple_obj = open_file("data/knapsack-test3.txt")
tuple_obj = open_file("data/knapsack-test4.txt")
# tuple_obj = open_file("data/knapsack.txt")
data_dict = tuple_obj[0]
knapsack_size = tuple_obj[1]
num_items = tuple_obj[2]
print(data_dict)
print(knapsack_size)
print(num_items)

A = []
for i in range(0, num_items + 1):
    A.append([])
    for j in range(0, knapsack_size + 1):
        A[i].append(0)
    
    
for i in range(1, num_items + 1):
    for j in range(0, knapsack_size + 1):
#         print(str(A[i-1][j]) +" vs "+ str(A[i-1][j-data_dict[i][1]]) + " + " + str(data_dict[i][0]))
        if data_dict[i][1] > j:
            A[i][j] = A[i-1][j]
        else:
            A[i][j] = max(A[i-1][j], A[i-1][j-data_dict[i][1]] + data_dict[i][0])
        
print(A[num_items][knapsack_size])

# 14
# 150
# 147
# 8
# 2493893

{1: (3, 4), 2: (2, 3), 3: (4, 2), 4: (4, 3)}
6
4
8


In [ ]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path -- location of file to read

    Returns:
    (data_dict, num_nodes) -- a tuple with a dictionary representing a graph and an integer reprsenting number of nodes
    """

    data_dict = {}
    knapsack_size = 0
    num_items = 0
    index = 1

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        knapsack_size = int(data_array[0].split(" ")[0])
        num_items = int(data_array[0].split(" ")[1])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            value = int(item.split(" ")[0])
            weight = int(item.split(" ")[1])
            data_dict[index] = (value, weight)
            index += 1
    return (data_dict, knapsack_size, num_items)


# tuple_obj = open_file("data/knapsack-test1.txt")
# tuple_obj = open_file("data/knapsack-test2.txt")
# tuple_obj = open_file("data/knapsack-test3.txt")
# tuple_obj = open_file("data/knapsack-test4.txt")
tuple_obj = open_file("data/knapsack-big.txt")
data_dict = tuple_obj[0]
knapsack_size = tuple_obj[1]
num_items = tuple_obj[2]
# print(data_dict)
# print(knapsack_size)
# print(num_items)

A = []
for i in range(0, 2):
    A.append([]) 
    for j in range(0, knapsack_size + 1):
        A[i].append(0)
    
i = 1
while i <= num_items:
    A[1][0:data_dict[i][1]] = A[0][0:data_dict[i][1]][:]
    for j in range(data_dict[i][1], knapsack_size + 1):
        if data_dict[i][1] > j:
            A[1][j] = A[0][j]
        else:
#             print(str(A[0][j]) +" vs "+ str(A[0][j-data_dict[i][1]]) + " + " + str(data_dict[i][0]))
            A[1][j] = max(A[0][j], A[0][j-data_dict[i][1]] + data_dict[i][0])
    A[0] = A[1][:] # copy array by value, not reference
    print(str(i) + " -> " + str(A[1][knapsack_size]))
    i += 1
     
# 14
# 150
# 147
# 8
# 4243395

1 -> 16808
2 -> 66882
3 -> 75813
4 -> 94427
5 -> 172351
6 -> 186718
7 -> 206770
8 -> 206770
9 -> 244745
10 -> 273207
11 -> 273207
12 -> 273207
13 -> 273207
14 -> 332193
15 -> 332193
16 -> 376508
17 -> 390021
18 -> 390021
19 -> 390021
20 -> 390021
21 -> 390021
22 -> 390021
23 -> 390021
24 -> 390021
25 -> 402671
26 -> 402671
27 -> 402671
28 -> 402671
29 -> 402671
30 -> 402671
31 -> 416881
32 -> 416881
33 -> 473835
34 -> 491250
35 -> 491250
36 -> 491250
37 -> 491250
38 -> 491250
39 -> 491250
40 -> 491250
41 -> 491250
42 -> 523483
43 -> 523483
44 -> 523483
45 -> 523483
46 -> 523483
47 -> 523483
48 -> 523483
49 -> 523483
50 -> 523483
51 -> 523483
52 -> 523483
53 -> 523483
54 -> 523483
55 -> 523483
56 -> 523483
57 -> 534272
58 -> 534272
59 -> 572669
60 -> 572669
61 -> 572669
62 -> 572669
63 -> 572669
64 -> 572669
65 -> 634721
66 -> 634721
67 -> 634721
68 -> 634721
69 -> 636272
70 -> 636272
71 -> 658913
72 -> 658913
73 -> 658913
74 -> 658913
75 -> 658913
76 -> 756886
77 -> 756886
78 -> 820021

## Sequence Alignment

- strings $X = x_{1} \dots x_{m}$, $Y = y_{1} \dots y_{m}$
- penalty $\alpha_{gap} \ge 0$ for inserting a gap, $\alpha_{ab}$ for matching $a$ and $b$
- insert gaps to equalize length of string

Final position of string can be one of
- case1: $x_{m}$ and $y_{n}$ matched
- case2: $x_{m}$ is matched with a gap
- case3: $y_{n}$ is matched with a gap

Let $X^{'} = X - x_{m}$ and $Y^{'} = Y - y_{m}$ 
- case1: alignment of $X^{'}$ and $Y^{'}$ is optimal
- case2: alignment of $X^{'}$ and $Y$ is optimal
- case3: alignment of $X$ and $Y^{'}$ is optimal

Subproblem $(X_{i}m Y_{j})$
- $X_{i}$ = 1st $i$ letters of $X$
- $Y_{j}$ = 1st $j$ letters of $Y$

Let $P_{ij}$ = penalty of optimal alignment of $X_{i}$ and $Y_{j}$
- For all i = 1 to n and j = 1 to n, $P_{ij}$ is the **minimun** of the following three cases
- case1: $\alpha_{x_{i}y_{j}}$ + $P_{i-1,j-1}$
- case2: $\alpha_{gap}$ + $P_{i-1,j}$
- case3: $\alpha_{gap}$ + $P_{i,j-1}$

Pseudo code
- let A = 2-D array
- $A[i,0] = A[0,j] = i * \alpha_{gap}$ for all $i \ge 0$
- for i = 1 to m
    - for j = 1 to n
        - $A[i,j]$ = $min\{A[i-1,j-1]+\alpha_{x_{i}y_{j}}, A[i-1,j]+\alpha_{gap}, A[i,j-1]+\alpha_{gap}\}$
        
Trace back through filled-in table $A_{i}$ starting at $A[m,n]$
- when reaching subproblem $A[i,j]$
    - if $A[i,j]$ filled using case1, match $x_{i}$ and $y_{j}$, and go to $A[i-1, j-1]$
    - if $A[i,j]$ filled using case2, match $x_{i}$ and a gap, and go to $A[i-1, j]$
    - if $A[i,j]$ filled using case3, match $y_{j}$ and a gap, and go to $A[i, j-1]$
- if $i=0$ or $j=0$, match remaining substring with gaps

## Optimal Binary Search Tree

- what is the best search tree for a given set of keys?
- let frequencies $p_{1} \dots p_{n}$ for items $1 \dots n$
- valid search tree that minimizes weighted search time

$C(T) = \displaystyle\sum_{i}P_{i}*$[search time for in i T]

- subtrees $T_{1}$ and $T_{2}$ are optimal BSTs for the keys $\{1 \dots r-1\}$ and $\{r+1 \dots n\}$
- for $1 \ge i \ge j \ge n$, let $C_{ij}$ = weighted search cost of optimal BST for items $\{i, i+1 \dots j-1, j\}$ with properties $\{p_{i}, p_{i+1} \dots p_{j}\}$
- for every $1 \ge i \ge j \ge n$

$C_{ij} = \underset{r=i}{\text{min}}\left[\displaystyle\sum_{k=1}^{j}P_{k}+C_{i,r-1}+C_{r+1,j}\right]$ where $C_{i,r-1}, C_{r+1,j} = 0$ if $x>y$

- let A=2-D array
- for s = 0 to n-1 (s represent j-i)
    - for i =1 to n
        - $A[i, i+s]$ = $\underset{r=i}{\text{min}}\left[\displaystyle\sum_{k=i}^{i+s}P_{k}+A[i,r-1]+A[r+1,i+s]\right]$ where $A[i,r-1]+A[r+1,i+s] = 0$ if first index $\ge$ second index
- return $A[1,n]$